In [5]:
import numpy as np
from scipy.interpolate import interp1d

class Interpolation:
    def __init__(self, frame_ids, coordinates, fps):
        self.frame_ids = np.array(frame_ids)
        self.coordinates = np.array(coordinates)
        self.fps = fps
        
        # Interpolators for x and y coordinates
        self.interpolator_x = interp1d(self.frame_ids, self.coordinates[:, 0], kind='linear', fill_value='extrapolate')
        self.interpolator_y = interp1d(self.frame_ids, self.coordinates[:, 1], kind='linear', fill_value='extrapolate')

    def value_in_frame(self, frame_id, seconds_sensitivity=1):
        frame_sensitivity = seconds_sensitivity * self.fps
        
        # Check if there are any frame IDs within the specified frame_sensitivity of the requested frame_id
        if not np.any((self.frame_ids >= frame_id - frame_sensitivity) & (self.frame_ids <= frame_id + frame_sensitivity)):
            return (np.nan, np.nan)
        
        x = self.interpolator_x(frame_id)
        y = self.interpolator_y(frame_id)
        return (x, y)

class InstrumentDistance:
    def __init__(self, frame_ids1, coordinates1, frame_ids2, coordinates2, fps):
        self.instrument1 = Interpolation(frame_ids1, coordinates1, fps)
        self.instrument2 = Interpolation(frame_ids2, coordinates2, fps)
        self.fps = fps
        
        # Determine the common range of frame_ids
        self.min_frame = max(min(frame_ids1), min(frame_ids2))
        self.max_frame = min(max(frame_ids1), max(frame_ids2))

    def average_distance(self, seconds_sensitivity=1):
        total_distance = 0
        valid_frame_count = 0

        for frame_id in range(self.min_frame, self.max_frame + 1):
            coord1 = self.instrument1.value_in_frame(frame_id, seconds_sensitivity)
            coord2 = self.instrument2.value_in_frame(frame_id, seconds_sensitivity)

            if not np.isnan(coord1).any() and not np.isnan(coord2).any():
                distance = np.linalg.norm(np.array(coord1) - np.array(coord2))
                total_distance += distance
                valid_frame_count += 1
        
        if valid_frame_count == 0:
            return np.nan  # No valid frames to calculate average distance
        
        return total_distance / valid_frame_count

    def frames_below_threshold(self, threshold, seconds_sensitivity=1):
        count = 0

        for frame_id in range(self.min_frame, self.max_frame + 1):
            coord1 = self.instrument1.value_in_frame(frame_id, seconds_sensitivity)
            coord2 = self.instrument2.value_in_frame(frame_id, seconds_sensitivity)

            if not np.isnan(coord1).any() and not np.isnan(coord2).any():
                distance = np.linalg.norm(np.array(coord1) - np.array(coord2))
                if distance < threshold:
                    count += 1
        
        return count

    def seconds_below_threshold(self, threshold, seconds_sensitivity=1):
        frame_count = self.frames_below_threshold(threshold, seconds_sensitivity)
        return frame_count / self.fps

# Example usage:
frame_ids1 = [1, 2, 4, 5]
coordinates1 = [(1, 1), (2, 2), (4, 4), (5, 5)]

frame_ids2 = [1, 3, 4, 6]
coordinates2 = [(2, 2), (3, 3), (4, 4), (6, 6)]

fps = 30  # Example frames per second

distance_calculator = InstrumentDistance(frame_ids1, coordinates1, frame_ids2, coordinates2, fps)
print("Average Distance:", distance_calculator.average_distance())
print("Frames below threshold 1.5:", distance_calculator.frames_below_threshold(1.5))
print("Seconds below threshold 1.5:", distance_calculator.seconds_below_threshold(1.5))


Average Distance: 0.42426406871192857
Frames below threshold 1.5: 5
Seconds below threshold 1.5: 0.16666666666666666


In [6]:
%pwd

'/webapps/piglegsurgery/devel'

In [4]:

ll = [1,2,3,4,5]

ll[None:None]

[1, 2, 3, 4, 5]